In [144]:
# Imports
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pygeocoder import Geocoder
from sklearn.preprocessing import StandardScaler
from geopy.geocoders import GeocodeFarm, Nominatim

In [7]:
clean_df = pd.read_csv("../data/cleaned_government_data.csv")

In [8]:
clean_df.head(5)

,Major Class,Month of Travel Date,From,To,Sum of Net Tickets,Sum of Total $
0,Business Class,Jan,Calgary,Ottawa,2,6045.62
1,Business Class,Jan,Calgary,Victoria,1,740.60
2,Business Class,Jan,Campbell River,Ottawa,1,3482.85
3,Business Class,Jan,Charlottetown,Calgary,1,2807.24
4,Business Class,Jan,Charlottetown,Ottawa,1,737.35


## Month of Travel Date

In [23]:
clean_df.groupby(["Month of Travel Date"])[["Sum of Net Tickets", "Sum of Total $"]].describe()

Sum of Net Tickets                                       \
                                  count       mean        std  min  25%  50%   
Month of Travel Date                                                           
Apr                              1322.0  14.500000  51.035518  1.0  1.0  3.0   
Aug                              1434.0  11.359833  30.533593  1.0  1.0  3.0   
Dec                              1096.0  12.875912  41.477783  1.0  1.0  3.0   
Feb                              1390.0  18.152518  66.668278  1.0  1.0  3.0   
Jan                              1303.0  16.861090  57.872933  1.0  1.0  3.0   
Jul                              1459.0  13.037012  35.379328  1.0  1.0  3.0   
Jun                              1492.0  17.484584  63.959791  1.0  1.0  3.0   
Mar                              1464.0  18.368852  66.263919  1.0  1.0  3.0   
May                              1489.0  18.243788  65.942885  1.0  1.0  3.0   
Nov                              1427.0  20.147162  77.934706  1.0  1.0  3.0   
Oct                              1432.0  19.288408  70.747471  1.0  1.0  3.0   
Sep                              1524.0  16.282152  59.851538  1.0  1.0  3.0   

                                   Sum of Total $                             \
                       75%     max          count         mean           std   
Month of Travel Date                                                           
Apr                    8.0  1059.0         1322.0  7564.120416  23791.913640   
Aug                    8.0   548.0         1434.0  5818.012629  14419.957700   
Dec                    8.0   776.0         1096.0  6074.260319  17666.768233   
Feb                    9.0  1426.0         1390.0  9000.705237  29700.126779   
Jan                    9.0  1071.0         1303.0  7889.833753  24820.491955   
Jul                    9.0   554.0         1459.0  6237.850206  16738.505276   
Jun                   10.0  1368.0         1492.0  8535.565449  27687.360080   
Mar                    9.0  1292.0         1464.0  9460.459713  32308.715377   
May                   10.0  1331.0         1489.0  8965.658784  28897.219857   
Nov                   10.0  1703.0         1427.0  9725.045452  35147.448876   
Oct                   10.0  1426.0         1432.0  9587.468869  32466.426523   
Sep                    9.0  1287.0         1524.0  7877.129403  25136.838466   

                                                                       
                        min       25%       50%        75%        max  
Month of Travel Date                                                   
Apr                    4.52  885.7600  1946.735  4970.7425  439706.32  
Aug                   22.60  840.5125  1871.980  4427.9000  229749.52  
Dec                   11.50  762.0200  1650.090  4350.1600  271952.43  
Feb                   16.95  867.6500  2060.060  5857.3050  530559.69  
Jan                   26.25  835.8700  1949.780  5296.0050  461784.07  
Jul                   19.00  844.7450  1818.220  5077.6800  358749.20  
Jun                   18.90  921.9725  2039.485  5455.4350  448261.71  
Mar                   17.25  872.9100  2022.590  5819.7800  639415.03  
May                   10.50  903.1500  1953.920  6013.5500  534445.88  
Nov                   33.90  834.5950  1957.820  6273.7650  671987.13  
Oct                   57.50  878.0825  2041.955  6086.2800  648160.39  
Sep                   10.50  835.6350  1920.090  5485.9775  453756.90

### Inference:
- One-hot encode months because there is no significant rise in price between the months.
- This was determined by diving the mean of Sum of Total with the Sum of Net Tickets for every month.

## Origin, Destination encoding

In [62]:
geo = Nominatim()

/Volumes/Macintosh HDD/Tech_Hackathon/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [63]:
geo_farm = GeocodeFarm()

In [174]:
cities=list(clean_df["From"].unique())
cities.extend(list(clean_df["To"].unique()))

In [ ]:
location_details = {}

In [170]:
for each_city in cities:
    try:
        location = geo.geocode(each_city + ", Canada")
    except:
        try:
            location = geo_farm.geocode(each_city + ", Canada")
        except:
            continue
    try:
        location_details[each_city] = {
            "latitude": location[1][0],
            "longitude": location[1][1]
        }
    except:
        continue

In [172]:
with open("../data/coordinates.json", "w") as f:
    json.dump(location_details, f)

In [171]:
clean_df[["From_lat","From_lon"]] = clean_df["From"].apply(lambda x: pd.Series([location_details[x]["latitude"],location_details[x]["longitude"]]))
clean_df[["To_lat","To_lon"]] = clean_df["To"].apply(lambda x: pd.Series([location_details[x]["latitude"],location_details[x]["longitude"]]))

In [178]:
clean_df.head(5)

,Major Class,Month of Travel Date,From,To,Sum of Net Tickets,Sum of Total $,From_lat,From_lon,To_lat,To_lon
0,Business Class,Jan,Calgary,Ottawa,2,6045.62,51.053423,-114.062589,45.421106,-75.690308
1,Business Class,Jan,Calgary,Victoria,1,740.60,51.053423,-114.062589,-36.598610,144.678005
2,Business Class,Jan,Campbell River,Ottawa,1,3482.85,50.023071,-125.244154,45.421106,-75.690308
3,Business Class,Jan,Charlottetown,Calgary,1,2807.24,46.234953,-63.132935,51.053423,-114.062589
4,Business Class,Jan,Charlottetown,Ottawa,1,737.35,46.234953,-63.132935,45.421106,-75.690308


,Major Class,Month of Travel Date,From,To,Sum of Net Tickets,Sum of Total $
0,Business Class,Jan,Calgary,Ottawa,2,6045.62
1,Business Class,Jan,Calgary,Victoria,1,740.60
2,Business Class,Jan,Campbell River,Ottawa,1,3482.85
3,Business Class,Jan,Charlottetown,Calgary,1,2807.24
4,Business Class,Jan,Charlottetown,Ottawa,1,737.35
...,...,...,...,...,...,...
16827,Premium Economy,Aug,Whitehorse,Ottawa,1,771.15
16828,Premium Economy,Aug,Williams Lake,Kingston,1,2786.89
16829,Premium Economy,Sep,Regina,Vancouver,1,1945.11
16830,Premium Economy,Oct,Penticton,Montreal,1,1513.46


## Major Class

In [31]:
clean_df.groupby(["Major Class"])[["Sum of Net Tickets", "Sum of Total $"]].sum()['Sum of Total $']/clean_df.groupby(["Major Class"])[["Sum of Net Tickets", "Sum of Total $"]].sum()['Sum of Net Tickets']

Major Class
Business Class     1845.703896
Economy             486.790989
First Class         235.342727
Premium Economy    1511.283000
dtype: float64

In [6]:
if __name__ == "__main__":
    main()

      Major Class Month of Travel Date            From        To  \
0  Business Class                  Jan         Calgary    Ottawa   
1  Business Class                  Jan         Calgary  Victoria   
2  Business Class                  Jan  Campbell River    Ottawa   
3  Business Class                  Jan   Charlottetown   Calgary   
4  Business Class                  Jan   Charlottetown    Ottawa   

   Sum of Net Tickets  Sum of Total $  
0                   2         6045.62  
1                   1          740.60  
2                   1         3482.85  
3                   1         2807.24  
4                   1          737.35  


In [33]:
clean_df[clean_df["Major Class"]=="First Class"]

,Major Class,Month of Travel Date,From,To,Sum of Net Tickets,Sum of Total $
16807,First Class,Jan,Vancouver,Toronto,2,303.58
16808,First Class,Jan,Victoria,Abbotsford,1,270.47
16809,First Class,May,Montreal,Quebec,1,231.74
16810,First Class,Aug,Lac Brochet,Thompson,2,695.10
16811,First Class,Sep,Vancouver,Victoria,1,207.00
16812,First Class,Sep,Victoria,Vancouver,1,159.00
16813,First Class,Nov,Vancouver,Nanaimo,1,108.00
16814,First Class,Nov,Winnipeg,St Theris Point,2,613.88
